In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


from transformers import T5Tokenizer, T5EncoderModel
import torch
import pandas as pd
import numpy as np
import re
import json
from tqdm.notebook import tqdm

import h5py

2024-01-23 16:09:19.784763: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 16:09:19.965732: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 16:09:19.999743: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-23 16:09:19.999760: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

# Load the model
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device)


In [3]:
protein_ref = pd.read_csv("../processed_data/mmseqs_searches/protein_sequences.csv")
protein_ref

,UniProt_ID,Protein_Sequence
0,A0A023HQ21,MEVLYSLSKTLKDARDKIVEGTLYSNVSDLIQQFNQMIVTMNGNDF...
1,A0A023IRJ5,MLLTEIPNVYQRALALERRIRVRELFPEGLVPYLKRQEHAGSIASP...
2,A0A023IRJ9,MASIIENLTTTFASSMLGTAEDAVGSLMDAAVRPVEAPNVIISDQS...
3,A0A023IT01,MASQGTKRSYEQMETGGERQDATEIRASVGRMIGGIGRFYIQMCTE...
4,A0A023IYH5,DYSKWDSTLHPDVIRLAIDTLSHFITLDDLAVGVNNVLSSRPSGLV...
...,...,...
128839,X6RLT1,XEGMAGAVPGAIMDEDYYGSAAEWGDEADGGQQEDDSGEGEDDAEV...
128840,X6RLV5,MSGYSSDRDRGRDRGFGAPRFGGSRAGPLSGKKFGNPGEKLVKKKW...
128841,X6RLX0,MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSV...
128842,X6RM00,MYGSARSVGKVEPSSQSPGRSPRLPRSPRLGHRRTNSTGGSSGSSV...


In [ ]:
prot_embeddings = []
for batch in tqdm(np.array_split(protein_ref["Protein_Sequence"].values, 12884)):
    lengths = [len(p) for p in batch]
    tok_batch = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in batch]
    ids = tokenizer.batch_encode_plus(tok_batch, add_special_tokens=True, padding="longest")

    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # generate embeddings
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,attention_mask=attention_mask)

    for i, l in enumerate(lengths):
        seq_emb = embedding_repr.last_hidden_state[i,:l].mean(dim=0)
        # idx += 1
        prot_embeddings.append(seq_emb.numpy())
        # dset[idx,:]=seq_emb.numpy()
    break

  0%|          | 0/12884 [00:00<?, ?it/s]

In [5]:
prot_embeddings = np.vstack(prot_embeddings)

50

In [ ]:
np.save("../processed_data/mmseqs_searches/prot_embeddings.npy", prot_embeddings)

In [6]:
# import numpy as np
# a = np.random.rand(128844, 2048)